In [23]:
import json
#from google.colab import files # solo para google collab
from google.cloud import storage
import pandas as pd
print(storage.__version__)

2.11.0


In [24]:
# google collab
#path_api_key = '/content/extreme-unison-399121-cadd77c555ca.json'

# entorno local
path_api_key = 'gcloud_api_key/extreme-unison-399121-cadd77c555ca.json'

# extraigo las credenciales para el acceso
service_account_info = json.load(open(path_api_key))

# autorizo el acceso a cloud storage
client_storage = storage.Client.from_service_account_info(service_account_info)

# carga de datos
# unicamente para .py en VM:
#data_gmaps_reviews_norm = pd.read_parquet('gs://1_transform/data_gmaps_reviews_norm.parquet')

# para entorno Google Colab o maquina local:

# cambiar segun bucket
bucket = client_storage.get_bucket('1_transform')

# cambiar segun dataframe
data_blob = bucket.blob('data_gmaps_reviews_norm.parquet')
data_gmaps_metadata = pd.read_parquet(storage.fileio.BlobReader(data_blob))
data_gmaps_metadata.info()
data_gmaps_metadata.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115400 entries, 0 to 1115399
Data columns (total 19 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   gmap_id          1115400 non-null  object 
 1   user_id          1115400 non-null  object 
 2   user_name        1115400 non-null  object 
 3   user_time        1115400 non-null  int64  
 4   rating           1115400 non-null  int64  
 5   user_text        685933 non-null   object 
 6   resp_time        181992 non-null   float64
 7   resp_text        181992 non-null   object 
 8   user_time_year   1115400 non-null  int32  
 9   user_time_month  1115400 non-null  int32  
 10  user_time_day    1115400 non-null  int32  
 11  user_time_hms    1115400 non-null  object 
 12  resp_time_year   181992 non-null   float64
 13  resp_time_month  181992 non-null   float64
 14  resp_time_day    181992 non-null   float64
 15  resp_time_hms    181992 non-null   object 
 16  state_name       1

,gmap_id,user_id,user_name,user_time,rating,user_text,resp_time,resp_text,user_time_year,user_time_month,user_time_day,user_time_hms,resp_time_year,resp_time_month,resp_time_day,resp_time_hms,state_name,state_code,zcta5_geoid
0,0x88892bb449d44cb3:0x7beaf0b2e3171a49,110771718750038252970,Christian Buckner,1630031374725,2,"No masks, requested curbside and waited 10 min...",NaN,None,2021,8,27,02:29:34.725000,NaN,NaN,NaN,None,alabama,AL,35115
1,0x88892bb449d44cb3:0x7beaf0b2e3171a49,104877410053821600291,Susie Smith,1602297776261,5,This location does an awesome job! The delive...,NaN,None,2020,10,10,02:42:56.261000,NaN,NaN,NaN,None,alabama,AL,35115
2,0x88892bb449d44cb3:0x7beaf0b2e3171a49,116879073472364348668,Brenda Brown,1598537775889,4,Best pizza I've had in a long time,NaN,None,2020,8,27,14:16:15.889000,NaN,NaN,NaN,None,alabama,AL,35115
3,0x88892bb449d44cb3:0x7beaf0b2e3171a49,109574390417810560616,Kam Str,1597615803053,3,None,NaN,None,2020,8,16,22:10:03.053000,NaN,NaN,NaN,None,alabama,AL,35115


In [25]:
data_gmaps_metadata.shape

(1115400, 19)

In [26]:
import os
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.sentiment.vader import SentimentIntensityAnalyzer

### 1. columna rating (0 a 5)>> promedio año-mes de rating >> rescalar de 0 a 1>> multiplicar por 0.4 = kpi_satisfaccion_rating por restaurante y año-mes (nueva columna).

In [27]:
# 1. Promedio de rating por año-mes
data_gmaps_metadata['user_time'] = pd.to_datetime(data_gmaps_metadata['user_time'], unit='ms')
data_gmaps_metadata['year_month'] = data_gmaps_metadata['user_time'].dt.to_period('M')
average_rating = data_gmaps_metadata.groupby('year_month')['rating'].mean().reset_index()
average_rating = average_rating.rename(columns={'rating': 'average_rating'})

# 2. Rescalar el rating promedio de 0 a 1
min_rating = average_rating['average_rating'].min()
max_rating = average_rating['average_rating'].max()
average_rating['rescaled_rating'] = (average_rating['average_rating'] - min_rating) / (max_rating - min_rating)

# 3. Calcular el KPI de satisfacción
average_rating['kpi_satisfaccion_rating'] = average_rating['rescaled_rating'] * 0.4

# 4. Unir los resultados con el DataFrame original
data_gmaps_metadata = data_gmaps_metadata.merge(average_rating, on='year_month', how='left')

# Ahora, 'data' contiene todas las columnas requeridas, incluyendo 'kpi_satisfaccion_rating'.

# Puedes guardar el DataFrame resultante en un archivo CSV si es necesario.
# data.to_csv('data_con_kpi.csv', index=False)

In [28]:
data_gmaps_metadata.head(2)

,gmap_id,user_id,user_name,user_time,rating,user_text,resp_time,resp_text,user_time_year,user_time_month,...,resp_time_month,resp_time_day,resp_time_hms,state_name,state_code,zcta5_geoid,year_month,average_rating,rescaled_rating,kpi_satisfaccion_rating
0,0x88892bb449d44cb3:0x7beaf0b2e3171a49,110771718750038252970,Christian Buckner,2021-08-27 02:29:34.725,2,"No masks, requested curbside and waited 10 min...",NaN,None,2021,8,...,NaN,NaN,None,alabama,AL,35115,2021-08,4.320092,0.332661,0.133064
1,0x88892bb449d44cb3:0x7beaf0b2e3171a49,104877410053821600291,Susie Smith,2020-10-10 02:42:56.261,5,This location does an awesome job! The delive...,NaN,None,2020,10,...,NaN,NaN,None,alabama,AL,35115,2020-10,4.320153,0.333731,0.133492


In [38]:
# Descarga los recursos necesarios de NLTK si no los tienes ya
nltk.download('vader_lexicon')

# Inicializar el analizador de sentimientos VADER
sia = SentimentIntensityAnalyzer()

# Función para clasificar el sentimiento basado en VADER
def classify_sentiment_vader(text):
    if pd.notna(text) and isinstance(text, str):
        sentiment_score = sia.polarity_scores(text)
        compound_score = sentiment_score['compound']
        return compound_score
    else:
        return None

# Calcular el valor de compound_score y agregarlo como columna
data_gmaps_metadata['compound_score'] = data_gmaps_metadata['user_text'].apply(classify_sentiment_vader)

# Calcular el promedio del compound_score por restaurante y año-mes
data_gmaps_metadata['year_month'] = pd.to_datetime(data_gmaps_metadata['user_time']).dt.to_period('M')
df_avg_compound = data_gmaps_metadata.groupby(['gmap_id', 'year_month'])['compound_score'].mean().reset_index()

# Rescalar de 0 a 1 y luego multiplicar por 0.6 para obtener el KPI de satisfacción de sentimiento
df_avg_compound['rescaled_score'] = (df_avg_compound['compound_score'] - df_avg_compound['compound_score'].min()) / (df_avg_compound['compound_score'].max() - df_avg_compound['compound_score'].min())
df_avg_compound['kpi_satisfaccion_sentimiento'] = df_avg_compound['rescaled_score'] * 0.6

# Suma de las dos columnas
df_avg_compound['sum_rating_compound_score'] = data_gmaps_metadata['average_rating'] + df_avg_compound['compound_score']

# Finalmente, df_avg_compound contendrá el KPI de satisfacción de sentimiento por restaurante y año-mes.


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Raksasa_pc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [39]:
df_avg_compound

,gmap_id,year_month,compound_score,rescaled_score,kpi_satisfaccion_sentimiento,sum_rating_compound_score
0,0x1469047418426e83:0x1ab10483596d44a4,2020-07,0.937700,0.969041,0.581425,5.257792
1,0x1469047418426e83:0x1ab10483596d44a4,2020-08,0.828200,0.914187,0.548512,5.148353
2,0x1469047418426e83:0x1ab10483596d44a4,2020-09,0.818183,0.909169,0.545501,5.129981
3,0x1469047418426e83:0x1ab10483596d44a4,2020-10,0.610350,0.805055,0.483033,4.922148
4,0x1469047418426e83:0x1ab10483596d44a4,2020-11,0.916700,0.958521,0.575113,5.236853
...,...,...,...,...,...,...
343906,0x89fca7af1a667519:0xcc610fc9e4e96bd3,2020-07,0.941500,0.970945,0.582567,5.261653
343907,0x89fca7af1a667519:0xcc610fc9e4e96bd3,2020-08,0.628750,0.814272,0.488563,4.929734
343908,0x89fca7af1a667519:0xcc610fc9e4e96bd3,2020-09,0.700300,0.850115,0.510069,5.021475
343909,0x89fca7af1a667519:0xcc610fc9e4e96bd3,2020-10,0.310200,0.654694,0.392816,4.621998


In [40]:
# Calcular el KPI de satisfacción de suma por restaurante y año-mes
df_avg_compound['kpi_satisfaccion_suma'] = data_gmaps_metadata['kpi_satisfaccion_rating'] + df_avg_compound['kpi_satisfaccion_sentimiento']
df_avg_compound.head(3)

,gmap_id,year_month,compound_score,rescaled_score,kpi_satisfaccion_sentimiento,sum_rating_compound_score,kpi_satisfaccion_suma
0,0x1469047418426e83:0x1ab10483596d44a4,2020-07,0.937700,0.969041,0.581425,5.257792,0.714489
1,0x1469047418426e83:0x1ab10483596d44a4,2020-08,0.828200,0.914187,0.548512,5.148353,0.682004
2,0x1469047418426e83:0x1ab10483596d44a4,2020-09,0.818183,0.909169,0.545501,5.129981,0.620807


In [43]:
# Fusiona los DataFrames 'df_avg_compound' y 'data_gmaps_metadata' en función de la columna 'gmap_id'
merged_df = pd.merge(df_avg_compound, data_gmaps_metadata, on='gmap_id', how='inner')

# Seleccionar las columnas requeridas después de la fusión
df_final_KPI_Satisfaccion = merged_df[['state_name','state_code','zcta5_geoid','gmap_id', 'user_name','year_month_x', 'kpi_satisfaccion_rating', 'kpi_satisfaccion_sentimiento','kpi_satisfaccion_suma', 'sum_rating_compound_score' ]]

# Dividir la columna 'year_month_x' en 'year' y 'month'
df_final_KPI_Satisfaccion[['year', 'month']] = df_final_KPI_Satisfaccion['year_month_x'].astype(str).str.split("-", expand=True)

# Eliminar la columna 'year_month_x'
df_final_KPI_Satisfaccion = df_final_KPI_Satisfaccion.drop(columns=['year_month_x'])



C:\Users\Raksasa_pc\AppData\Local\Temp\ipykernel_1112\111187548.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_KPI_Satisfaccion[['year', 'month']] = df_final_KPI_Satisfaccion['year_month_x'].astype(str).str.split("-", expand=True)
C:\Users\Raksasa_pc\AppData\Local\Temp\ipykernel_1112\111187548.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_KPI_Satisfaccion[['year', 'month']] = df_final_KPI_Satisfaccion['year_month_x'].astype(str).str.split("-", expand=True)


In [44]:
df_final_KPI_Satisfaccion.head()

,state_name,state_code,zcta5_geoid,gmap_id,user_name,kpi_satisfaccion_rating,kpi_satisfaccion_sentimiento,kpi_satisfaccion_suma,year,month
0,florida,FL,33776,0x1469047418426e83:0x1ab10483596d44a4,Olga Kovalenko,0.182659,0.581425,0.714489,2020,07
1,florida,FL,33776,0x1469047418426e83:0x1ab10483596d44a4,Anthony Scafidi,0.140606,0.581425,0.714489,2020,07
2,florida,FL,33776,0x1469047418426e83:0x1ab10483596d44a4,Trisha McCabe,0.133492,0.581425,0.714489,2020,07
3,florida,FL,33776,0x1469047418426e83:0x1ab10483596d44a4,Carla Billeter,0.000000,0.581425,0.714489,2020,07
4,florida,FL,33776,0x1469047418426e83:0x1ab10483596d44a4,Jennifer Murphy,0.142981,0.581425,0.714489,2020,07


In [45]:
# Guardar el DataFrame final como un archivo parquet
df_final_KPI_Satisfaccion.to_parquet('gmaps_kpi_satisfaccion.parquet', index=False)